# Gaussian manifold analytic validation

Purpose: validate geodesics and NGD on Gaussian statistical manifolds per the spec in CFT+ULCC Framework Implementation.pdf.

Mean-only manifold (fixed covariance Σ): Fisher–Rao metric for mean parameters is constant g=Σ^{-1}. FR distance: d_FR(μ0, μ1) = sqrt((μ1−μ0)^T Σ^{-1} (μ1−μ0)). Geodesics are straight lines in μ since Christoffel symbols vanish.

SPD covariance manifold (optional): AIRM distance d_AIRM(Σ0,Σ1) = ||log(Σ0^{-1/2} Σ1 Σ0^{-1/2})||_F and closed-form geodesic Σ(t)=Σ0^{1/2} exp(t log(Σ0^{-1/2} Σ1 Σ0^{-1/2})) Σ0^{1/2}.

We compare numeric integrators to these analytics and enforce strict tolerances.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

np.set_printoptions(precision=6, suppress=True)

fig_dir = Path("spec-first/artifacts/figures")
fig_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Mean-only Gaussian geometry: covariance and Fisher metric
Sigma = np.array([[0.5, 0.1],
                  [0.1, 0.3]], dtype=float)
g = np.linalg.inv(Sigma)

def fr_distance_mean(mu0, mu1, g=g):
    """Fisher–Rao distance under constant metric g=Sigma^{-1}: sqrt((mu1-mu0)^T g (mu1-mu0))."""
    d = mu1 - mu0
    return float(np.sqrt(d.T @ g @ d))

def ngd_flow_mean(mu0, mu_star, t, rate=1.0):
    """Analytic NGD flow for quadratic potential 0.5 (mu-mu*)^T g (mu-mu*): mu(t)=mu*+(mu0-mu*) e^{-rate t}."""
    return mu_star + (mu0 - mu_star) * np.exp(-rate * t)

In [ ]:
# Numeric geodesic in mean manifold (Christoffel=0 => mü=0)
mu0 = np.array([-0.6,  0.2], dtype=float)
mu1 = np.array([ 0.9, -0.4], dtype=float)
T = 1.0
N = 2000
dt = T / N
times = np.linspace(0.0, T, N+1)

v = (mu1 - mu0) / T
mu_num = np.empty((N+1, 2), dtype=float)
mu_num[0] = mu0
for n in range(N):
    mu_num[n+1] = mu_num[n] + v * dt

mu_an = mu0 + np.outer(times / T, (mu1 - mu0))

diffs = mu_num - mu_an
max_abs_err = float(np.max(np.linalg.norm(diffs, axis=1)))

# Fisher–Rao path length
L_numeric = 0.0
for n in range(N):
    dmu = mu_num[n+1] - mu_num[n]
    L_numeric += float(np.sqrt(dmu.T @ g @ dmu))
L_analytic = fr_distance_mean(mu0, mu1, g)
rel_len_err = abs(L_numeric - L_analytic) / L_analytic

# Plot components
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(times, mu_num[:, 0], '-',  label='mu[0] numeric')
ax.plot(times, mu_an[:, 0],  '--', label='mu[0] analytic')
ax.plot(times, mu_num[:, 1], '-',  label='mu[1] numeric')
ax.plot(times, mu_an[:, 1],  '--', label='mu[1] analytic')
ax.set_xlabel('t')
ax.set_ylabel('mu components')
ax.set_title('Mean-manifold geodesic: numeric vs analytic')
ax.legend(loc='best', fontsize=8)
fig.tight_layout()

eod = fig_dir / 'gaussian_mean_geodesic.png'
fig.savefig(eod, dpi=150)
plt.close(fig)
geod_fig_path = eod

In [ ]:
# Natural Gradient Descent (NGD) on mean manifold for quadratic potential
mu_star = np.array([0.2, -0.1], dtype=float)
mu0_ngd = np.array([-0.6,  0.2], dtype=float)
T_ngd = 2.0
N_ngd = 4000
lr = T_ngd / N_ngd

times_ngd = np.linspace(0.0, T_ngd, N_ngd+1)
mu_ngd = np.empty((N_ngd+1, 2), dtype=float)
mu_ngd[0] = mu0_ngd
for n in range(N_ngd):
    # Discrete NGD Euler: mu_{n+1} = mu_n - lr * g^{-1} grad V = mu_n - lr*(mu_n - mu_star)
    mu_ngd[n+1] = mu_ngd[n] - lr * (mu_ngd[n] - mu_star)

mu_an_ngd = mu_star + np.outer(np.exp(-times_ngd), (mu0_ngd - mu_star))
mu_an_final = mu_star + (mu0_ngd - mu_star) * np.exp(-T_ngd)
mu_num_final = mu_ngd[-1]
rel_err_ngd = float(np.linalg.norm(mu_num_final - mu_an_final) / max(1e-12, np.linalg.norm(mu_an_final)))

# Plot
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(times_ngd, mu_ngd[:, 0], '-',  label='mu[0] NGD numeric')
ax.plot(times_ngd, mu_an_ngd[:, 0],  '--', label='mu[0] NGD analytic')
ax.plot(times_ngd, mu_ngd[:, 1], '-',  label='mu[1] NGD numeric')
ax.plot(times_ngd, mu_an_ngd[:, 1],  '--', label='mu[1] NGD analytic')
ax.set_xlabel('t')
ax.set_ylabel('mu components')
ax.set_title('Mean-manifold NGD: numeric vs analytic')
ax.legend(loc='best', fontsize=8)
fig.tight_layout()

nfp = fig_dir / 'gaussian_mean_ngd.png'
fig.savefig(nfp, dpi=150)
plt.close(fig)
ngd_fig_path = nfp

In [ ]:
# Optional: SPD 2x2 covariance manifold with AIRM
_eps = 1e-12

def matsqrt(A):
    w, V = np.linalg.eigh(A)
    w = np.clip(w, _eps, None)
    return V @ (np.diag(np.sqrt(w))) @ V.T

def matinv_sqrt(A):
    w, V = np.linalg.eigh(A)
    w = np.clip(w, _eps, None)
    return V @ (np.diag(1.0/np.sqrt(w))) @ V.T

def matlog(A):
    w, V = np.linalg.eigh(A)
    w = np.clip(w, _eps, None)
    return V @ (np.diag(np.log(w))) @ V.T

def matexp(A):
    w, V = np.linalg.eigh(A)
    return V @ (np.diag(np.exp(w))) @ V.T

def airm_distance(S0, S1):
    S0_inv_half = matinv_sqrt(S0)
    inner = S0_inv_half @ S1 @ S0_inv_half
    L = matlog(inner)
    return float(np.linalg.norm(L, 'fro'))

def airm_geodesic(S0, S1, t):
    S0_half = matsqrt(S0)
    S0_inv_half = matinv_sqrt(S0)
    inner = S0_inv_half @ S1 @ S0_inv_half
    return S0_half @ matexp(t * matlog(inner)) @ S0_half

# SPD endpoints
S0 = np.array([[1.0,  0.2],
               [0.2,  0.5]], dtype=float)
S1 = np.array([[0.7, -0.1],
               [-0.1, 1.2]], dtype=float)
S0 = 0.5 * (S0 + S0.T)
S1 = 0.5 * (S1 + S1.T)

L_analytic_spd = airm_distance(S0, S1)

N_spd = 200
tgrid = np.linspace(0.0, 1.0, N_spd+1)
Sigmas = [airm_geodesic(S0, S1, float(t)) for t in tgrid]

L_numeric_spd = 0.0
sym_errs = []
for i in range(N_spd):
    A = Sigmas[i]
    B = Sigmas[i+1]
    sym_errs.append(np.linalg.norm(A - A.T, 'fro'))
    L_numeric_spd += airm_distance(A, B)
sym_errs.append(np.linalg.norm(Sigmas[-1] - Sigmas[-1].T, 'fro'))

rel_len_err_spd = abs(L_numeric_spd - L_analytic_spd) / L_analytic_spd
sym_err_max = float(np.max(sym_errs))

# Plot eigenvalues along geodesic
vals = np.array([np.linalg.eigvalsh(S) for S in Sigmas])
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(tgrid, vals[:, 0], '-', label='eigval 1')
ax.plot(tgrid, vals[:, 1], '-', label='eigval 2')
ax.set_xlabel('t')
ax.set_ylabel('eigenvalues')
ax.set_title('SPD 2x2 AIRM geodesic eigenvalues')
ax.legend(loc='best', fontsize=8)
fig.tight_layout()

sfp = fig_dir / 'gaussian_spd_geodesic.png'
fig.savefig(sfp, dpi=150)
plt.close(fig)
spd_fig_path = sfp

In [ ]:
# Results summary and acceptance checks
print('Mean-manifold geodesic max_abs_err:', max_abs_err)
print('FR length analytic vs numeric:', L_analytic, L_numeric, 'rel_err=', rel_len_err)
print('NGD relative error at final time:', rel_err_ngd)
print('SPD (AIRM) length analytic vs numeric:', L_analytic_spd, L_numeric_spd, 'rel_err=', rel_len_err_spd)
print('SPD symmetry max error:', sym_err_max)
print('Artifacts:')
print(' -', geod_fig_path)
print(' -', ngd_fig_path)
print(' -', spd_fig_path)

assert max_abs_err < 1e-6
assert abs(L_numeric - L_analytic) / L_analytic < 1e-6
assert rel_err_ngd < 1e-3
assert sym_err_max < 1e-12
assert rel_len_err_spd < 1e-3